In [21]:
import json
from copy import copy

import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [22]:
# import plotly.io as pio
# pio.renderers.default = 'browser'

In [23]:
def read_json_file(filepath: str, label: str) -> pd.DataFrame:
    with open(filepath) as json_file:
        data = json.load(json_file)
    new_data = {}
    row_number = 1
    for generation, values in data.items():
        for individual, individual_values in values.items():
            if "agg_fitness" in individual_values.keys():
                new_data.update({str(row_number): {"Generation": int(generation), "population_number": int(individual), 
                                           "agg_fitness": individual_values["agg_fitness"],
                                           "cost": individual_values["fitness"][3],
                                           "throughput_time": individual_values["fitness"][2],
                                           "throughput": individual_values["fitness"][0],
                                           "wip": individual_values["fitness"][1],
                                            "time": individual_values["time_stamp"]
                                          }})
                row_number += 1
    df = pd.DataFrame(new_data)
    df = df.T
    df["optimizer"] = label
    if label == 'anneal':
        df['agg_fitness'] = -1 * df['agg_fitness']
    df['agg_fitness'] = 0.004 * df['throughput'] - 1.0*df['wip'] - 0.0003 * df['cost']
    return df.copy()

In [24]:
def is_pareto_efficient_simple(costs):
    """
    Find the pareto-efficient points
    :param costs: An (n_points, n_costs) array
    :return: A (n_points, ) boolean array, indicating whether each point is Pareto efficient
    """
    is_efficient = np.ones(costs.shape[0], dtype = bool)
    for i, c in enumerate(costs):
        if is_efficient[i]:
            is_efficient[is_efficient] = np.any(costs[is_efficient]<c, axis=1)  # Keep any point with a lower cost
            is_efficient[i] = True  # And keep self
    return is_efficient

# Faster than is_pareto_efficient_simple, but less readable.
def is_pareto_efficient(costs, return_mask = True):
    """
    Find the pareto-efficient points
    :param costs: An (n_points, n_costs) array
    :param return_mask: True to return a mask
    :return: An array of indices of pareto-efficient points.
        If return_mask is True, this will be an (n_points, ) boolean array
        Otherwise it will be a (n_efficient_points, ) integer array of indices.
    """
    is_efficient = np.arange(costs.shape[0])
    n_points = costs.shape[0]
    next_point_index = 0  # Next index in the is_efficient array to search for
    while next_point_index<len(costs):
        nondominated_point_mask = np.any(costs<costs[next_point_index], axis=1)
        nondominated_point_mask[next_point_index] = True
        is_efficient = is_efficient[nondominated_point_mask]  # Remove dominated points
        costs = costs[nondominated_point_mask]
        next_point_index = np.sum(nondominated_point_mask[:next_point_index])+1
    if return_mask:
        is_efficient_mask = np.zeros(n_points, dtype = bool)
        is_efficient_mask[is_efficient] = True
        return is_efficient_mask
    else:
        return is_efficient

In [25]:
def get_performance_filtered_df(df) -> pd.DataFrame:
    df = df.copy()
    df = df.loc[df["wip"] < 150]
    df = df.loc[df["throughput"] > 2500]
    df = df.loc[df["throughput_time"] < 100]
    return df

In [26]:
anneal = "anneal_results.json"
tabu = "tabu_results.json"
ea ="ea_results.json"

dfs = []
dfs.append(read_json_file(anneal, "Anneal"))
dfs.append(read_json_file(ea, "NSGA2"))
dfs.append(read_json_file(tabu, "Tabu"))
df = pd.concat(dfs)

In [27]:
df["cost"] = df["cost"].astype(float)
df["agg_fitness"] = df["agg_fitness"].astype(float)

In [28]:
df = df.loc[df['time'] <= 10000]

In [29]:
# df_filtered = df.drop_duplicates(subset=['agg_fitness', 'cost', 'throughput', 'throughput_time', 'wip', 'optimizer']).copy()
df_filtered = df.drop_duplicates(subset=['agg_fitness', 'cost', 'throughput', 'throughput_time', 'wip']).copy()

In [30]:
df_filtered = get_performance_filtered_df(df_filtered)

In [31]:
columns = ['cost', 'throughput', 'wip']
# columns = ['throughput_time', 'throughput']
df_for_pareto = df_filtered[columns].copy()
df_for_pareto['throughput'] = -df_for_pareto['throughput']

In [32]:
is_efficient = is_pareto_efficient(df_for_pareto.values)
df_filtered['is_efficient'] = is_efficient

In [33]:
df_filtered['is_efficient'].sum()

65

In [34]:
df_efficient = df_filtered.loc[df_filtered['is_efficient']]
columns = ['agg_fitness', 'cost', 'throughput_time', 'throughput', 'wip', 'is_efficient']
df_efficient = df_efficient[columns]
# df_efficient.rename(columns={'is_efficient': 'red_is_efficient'}, inplace=True)

In [35]:
df = pd.merge(df, df_efficient, how='left')
df.fillna(False, inplace=True)

In [36]:
df['is_efficient'].sum()

138

In [37]:
# efficient = df.loc[df['is_efficient']].drop_duplicates(subset=['wip', 'cost', 'throughput'])
# c = efficient['cost'].mean()
# t = efficient['throughput'].mean()
# w = efficient['wip'].mean()
# print(c, t, w)
# throughput_rate = w / t
# throughput_rate
# cost_weight = w / c
# cost_weight
# print(throughput_rate, cost_weight)

In [38]:
def hex_to_rgba(h, alpha):
    '''
    converts color value in hex format to rgba format with alpha transparency
    '''
    return tuple([int(h.lstrip('#')[i:i+2], 16) for i in (0, 2, 4)] + [alpha])

In [39]:
def add_scatter(fig, time, mean_values, std_values, max_values, all_values, colors, label):
    color = colors.pop(0)
    fig.add_scatter(
    name=label + " mean fitness",
    x=time,
    y=mean_values,
    mode="lines",
    line=dict(color=color)
            )
    fig.add_scatter(
        name=label + " max fitness",
        x=time,
        y=max_values,
        mode="lines",
        line=dict(color=color, dash='dash')
    )
    # fig.add_scatter(
    #     name=label,
    #     x=time,
    #     y=all_values,
    #     mode = 'markers',
    #     opacity=0.5,
    #     marker=dict(
    #         color=color,
    #         size=2
    #     )    
    # )
    fig.update_layout(template="simple_white")
    fig.update_layout(
    # title="Plot Title",
    yaxis_title="fitness",
    xaxis_title="time in minutes",
    font=dict(
        family="Arial, monospace",
        # size=18,
        # color="RebeccaPurple"
        )
    )


In [40]:
grouped = df.loc[df['agg_fitness'] > -50000].groupby(by=['optimizer'])
mean_values = grouped.rolling(200)['agg_fitness'].mean()
std_values = grouped.rolling(200)['agg_fitness'].std()
max_values = grouped['agg_fitness'].cummax()
max_values.index = std_values.index
mean_values = grouped.rolling(200)['agg_fitness'].mean()
all_values = grouped.rolling(1)['agg_fitness'].mean()
time = grouped['time'].rolling(1).mean()
time = time / 60

In [41]:
fig = go.Figure()        
colors = copy(px.colors.qualitative.Plotly)
for optimizer in df['optimizer'].unique():
    add_scatter(fig, time[optimizer], mean_values[optimizer], std_values[optimizer], max_values[optimizer], all_values[optimizer], colors, label=optimizer)
fig.update_layout(height=350)
fig.show()
print("starting save")
fig.write_image("C:/Users/Sebas/OneDrive/Documents/prodsim/data/images/fitness_over_time.png")
print("saved")

starting save


In [ ]:
def plot_performance_2d(df):
    fig = px.scatter(df, x='throughput', y='wip',
                color=df['cost'], 
                # color_continuous_scale=px.colors.sequential.Viridis,
                color_continuous_scale=px.colors.sequential.Darkmint,                
                facet_col="optimizer",
                labels={
                     "throughput": "Throughput in pices",
                     "wip": "WIP in pieces",
                 },
    )
    fig.for_each_annotation(lambda a: a.update(text=a.text.replace("optimizer=", "")))
    fig.update_layout(coloraxis_colorbar=dict(
        title="Cost",
        # thicknessmode="pixels", thickness=50,
        # lenmode="pixels", len=200,
        # yanchor="top", y=1,
        # ticks="outside", ticksuffix=" bills",
        # dtick=5
    ))
    fig.update_layout(template="simple_white")
    fig.update_layout(
        font=dict(
            family="Arial, monospace",
            # size=18,
            # color="RebeccaPurple"
            )
    )
    fig.show()
    fig.write_image("images/performances.svg")


In [ ]:
df_perf_filtered = get_performance_filtered_df(df)
plot_performance_2d(df_perf_filtered)

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


In [ ]:
def mesh3d(df, efficient_df):
    df = df.drop_duplicates(subset=['agg_fitness', 'cost', 'throughput', 'throughput_time', 'wip']).copy()
    efficient_df = efficient_df.drop_duplicates(subset=['agg_fitness', 'cost', 'throughput', 'throughput_time', 'wip']).copy()

    data = []

    x2 = efficient_df['cost']
    y2 = efficient_df['throughput']
    z2 = efficient_df['wip']
    intensity = efficient_df['agg_fitness']

    data.append(go.Mesh3d(x=x2, y=y2, z=z2, color='grey', intensity=intensity, colorscale="Darkmint", opacity=0.95))

    df_non_efficient = df.loc[~df['is_efficient']]
    x = df_non_efficient['cost']
    y = df_non_efficient['throughput']
    z = df_non_efficient['wip']

    data.append(go.Scatter3d(x=x, y=y, z=z,
                                       mode='markers', marker=dict(
            size=5,
            color='grey',
            opacity=0.3,
        ))
    )

    df_non_efficient = df.loc[df['is_efficient']]
    x = df_non_efficient['cost']
    y = df_non_efficient['throughput']
    z = df_non_efficient['wip']

    data.append(go.Scatter3d(x=x, y=y, z=z,
                                       mode='markers', marker=dict(
            size=5,
            color='blue',
        ))
    )


    # data.reverse()
    fig = go.Figure(data=data)
    camera = dict(
        up=dict(x=1, y=0, z=0.5),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=1.25, y=-1.25, z=1.25)
    )

    fig.update_layout(scene_camera=camera)
    fig.update_layout(
        scene = dict(
            xaxis = dict(nticks=4, range=[-1000,100000],),
            yaxis = dict(nticks=4, range=[3200,3500],),
            zaxis = dict(nticks=4, range=[0,40],),
            xaxis_title='Cost',
            yaxis_title='Thoughput',
            zaxis_title='WIP',
        ),
        width=650,
        # height=400,
        margin=dict(r=10, l=10, b=10, t=10)
    )
    fig.update_layout(template="simple_white")
    fig.update_layout(
        font=dict(
            family="Arial, monospace",
            # size=18,
            # color="RebeccaPurple"
            )
    )
    fig.update_layout(showlegend=False)
    return fig

In [ ]:
df_efficient = df.loc[df['is_efficient']]

# mesh3d(df, df_efficient)
fig = mesh3d(df.loc[df['optimizer'] == 'NSGA2'], df_efficient)
fig.show()
fig.write_image("images/3d_NSGA2.svg")

fig = mesh3d(df.loc[df['optimizer'] == 'Anneal'], df_efficient)
fig.show()
fig.write_image("images/3d_Anneal.svg")

fig = mesh3d(df.loc[df['optimizer'] == 'Tabu'], df_efficient)
fig.show()
fig.write_image("images/3d_Tabu.svg")

In [ ]:
df_filtered = df.drop_duplicates(subset=['agg_fitness', 'cost', 'throughput', 'throughput_time', 'wip', 'optimizer']).copy()
is_efficient_cum_sum = df_filtered.groupby(by='optimizer')['is_efficient'].cumsum()
df_filtered['is_efficient_cum_sum'] = is_efficient_cum_sum

In [ ]:
def plot_performance_2d(df):
    df = df.copy()
    df['time'] = df['time'] / 60
    fig = px.line(df, x='time', y='is_efficient_cum_sum', color=df['optimizer'],
                    labels={
                        "time": "Time in minutes",
                        "is_efficient_cum_sum": "Number of pareto optimal solutions",
                    },
    )
    fig.update_layout(template="simple_white")
    fig.update_layout(
        font=dict(
            family="Arial, monospace",
            # size=18,
            # color="RebeccaPurple"
            ),
        legend_title="",
        height=350
    )
    
    fig.show()
    fig.write_image("images/pareto_numbers.svg")

In [ ]:
plot_performance_2d(df_filtered)